In [254]:
import pandas as pd
import numpy as np
import sys
import math
import csv
import urllib
import collections
import sklearn

from sklearn import datasets
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import cross_validate 
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn import linear_model
from sklearn import tree
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report

In [255]:
EPL_data = pd.read_csv('EPL.csv')

EPL_data.head()

,ID,Year,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,HF,AF,HC,AC,HY,AY,HR,AR,Winner,Loser
0,1.0,2019.0,09-08-19,Liverpool,Norwich,4.0,1.0,H,4.0,0.0,...,9.0,9.0,11.0,2.0,0.0,2.0,0.0,0.0,Liverpool,Norwich
1,2.0,2019.0,10-08-19,West Ham,Man City,0.0,5.0,A,0.0,1.0,...,6.0,13.0,1.0,1.0,2.0,2.0,0.0,0.0,Man City,West Ham
2,3.0,2019.0,10-08-19,Bournemouth,Sheffield United,1.0,1.0,D,0.0,0.0,...,10.0,19.0,3.0,4.0,2.0,1.0,0.0,0.0,None,None
3,4.0,2019.0,10-08-19,Burnley,Southampton,3.0,0.0,H,0.0,0.0,...,6.0,12.0,2.0,7.0,0.0,0.0,0.0,0.0,Burnley,Southampton
4,5.0,2019.0,10-08-19,Crystal Palace,Everton,0.0,0.0,D,0.0,0.0,...,16.0,14.0,6.0,2.0,2.0,1.0,0.0,1.0,None,None


In [256]:
team_names = pd.read_csv('teams.csv')
print(team_names)

           Team_Name
0            Arsenal
1        Aston Villa
2        Bournemouth
3           Brighton
4            Burnley
5            Chelsea
6     Crystal Palace
7            Everton
8          Leicester
9          Liverpool
10          Man City
11        Man United
12         Newcastle
13           Norwich
14  Sheffield United
15       Southampton
16         Tottenham
17           Watford
18          West Ham
19            Wolves


In [257]:
teamList = team_names['Team_Name'].tolist()
team_names.tail()

,Team_Name
15,Southampton
16,Tottenham
17,Watford
18,West Ham
19,Wolves


In [258]:
team_names.head()


,Team_Name
0,Arsenal
1,Aston Villa
2,Bournemouth
3,Brighton
4,Burnley


In [259]:
print(team_names[team_names['Team_Name']=='Arsenal'])

  Team_Name
0   Arsenal


In [260]:
print(EPL_data.head())


    ID    Year      Date        HomeTeam          AwayTeam  FTHG  FTAG FTR  \
0  1.0  2019.0  09-08-19       Liverpool           Norwich   4.0   1.0   H   
1  2.0  2019.0  10-08-19        West Ham          Man City   0.0   5.0   A   
2  3.0  2019.0  10-08-19     Bournemouth  Sheffield United   1.0   1.0   D   
3  4.0  2019.0  10-08-19         Burnley       Southampton   3.0   0.0   H   
4  5.0  2019.0  10-08-19  Crystal Palace           Everton   0.0   0.0   D   

   HTHG  HTAG  ...    HF    AF    HC   AC   HY   AY   HR   AR     Winner  \
0   4.0   0.0  ...   9.0   9.0  11.0  2.0  0.0  2.0  0.0  0.0  Liverpool   
1   0.0   1.0  ...   6.0  13.0   1.0  1.0  2.0  2.0  0.0  0.0   Man City   
2   0.0   0.0  ...  10.0  19.0   3.0  4.0  2.0  1.0  0.0  0.0       None   
3   0.0   0.0  ...   6.0  12.0   2.0  7.0  0.0  0.0  0.0  0.0    Burnley   
4   0.0   0.0  ...  16.0  14.0   6.0  2.0  2.0  1.0  0.0  1.0       None   

         Loser  
0      Norwich  
1     West Ham  
2         None  
3  Sou

In [261]:
def getAnnualTeamData(teamName, year):
    
    annual_data = EPL_data[EPL_data['Year'] == year]
    
    #number of goals scored in wins and losses
    gamesHome = annual_data[annual_data['HomeTeam'] == teamName]
    totalGoalsScored = gamesHome['FTHG'].sum()
    gamesAway = annual_data[annual_data['AwayTeam'] == teamName]
    totalGames = gamesHome.append(gamesAway)
    numGames = len(totalGames.index)
    
    #total goals scored
    totalGoalsScored += gamesAway['FTAG'].sum()
    
    #total goals allowed
    
    totalGoalsAllowed = gamesHome['FTAG'].sum()
    totalGoalsAllowed += gamesAway['FTHG'].sum()
    
    #discipline: total red cards, total yellow cards
    totalYellowCards = gamesHome['HY'].sum()
    totalYellowCards += gamesAway['AY'].sum()
    totalRedCards = gamesHome['HR'].sum()
    totalRedCards += gamesAway['AR'].sum()
   
    #total fouls
    totalFouls = gamesHome['HF'].sum()
    totalFouls += gamesAway['AF'].sum()
    
    #total Corners
    totalCorners = gamesHome['HC'].sum()
    totalCorners += gamesAway['AC'].sum()
    
    #shots per game = total Shots/total games
    totalShots = gamesHome['HS'].sum()
    totalShots += gamesAway['AS'].sum()
    
    if numGames != 0:
        spg = totalShots / numGames
        
        
    # avg shots allowed per game
    totalShotsAgainst = gamesHome['AS'].sum()
    totalShotsAgainst += gamesAway['HS'].sum()
    if numGames != 0:
        sag = totalShotsAgainst / numGames

    #Games Won Percentage = Games Won / (Games Won + Games Lost) 
    gamesWon = annual_data[annual_data['Winner'] == teamName] 
    gamesLost = annual_data[annual_data['Loser'] == teamName] 
    numGamesWon = len(gamesWon.index)
    numGamesLost = len(gamesLost.index)
    if numGames != 0:
        gamesWonPercentage = numGamesWon / numGames
    
    #Defense stats
        #Goalie Saves = Shots on Goal - Goal Scored
    totalShotsOnGoal = gamesHome['HST'].sum()
    totalShotsOnGoal += gamesAway['AST'].sum()
    goalieSaves = totalShotsOnGoal - totalGoalsAllowed
    
    #Saves Percentage = Goalie Saves / Shots on Goal   
    if totalShotsOnGoal != 0:
        savesPercentage = goalieSaves / totalShotsOnGoal
        
    #Saves Ratio = Shots On Goal / Goalie Saves    
    if goalieSaves != 0:
        savesRatio = totalShotsOnGoal / goalieSaves

    #Offense stats
    #Scoring Percentage = (Scoring Attempts - Goals Scored ) / Scoring Attempts
    if totalShots != 0:
        scoringPercentage = (totalShots - totalGoalsScored) / totalShots
        
    #Scoring Ratio = Shots On Goal / Goals Scored
    if totalGoalsScored != 0:
        scoringRatio = totalShotsOnGoal / totalGoalsScored       
        
            
    if numGames == 0: #if team not in dataset
        gamesWon = 0
        gamesLost = 0
        totalGoalsScored = 0
        totalGoalsAllowed = 0
        totalYellowCards = 0
        totalRedCards = 0
        totalFouls = 0
        totalCorners = 0
        spg = 0
        sag = 0
        gamesWonPercentage = 0
        goalieSaves = 0
        savesPercentage = 0
        savesRatio = 0
        scoringPercentage = 0
        scoringRatio = 0 
        
    return [totalGoalsScored, totalGoalsAllowed, totalYellowCards, totalRedCards,
        totalFouls,totalCorners, spg, sag, gamesWonPercentage, goalieSaves, savesPercentage, savesRatio,
        scoringPercentage, scoringRatio]
        

In [262]:
getAnnualTeamData('Sheffield United', 2019)

[30.0,
 25.0,
 51.0,
 1.0,
 303.0,
 166.0,
 10.25,
 11.321428571428571,
 0.39285714285714285,
 68.0,
 0.7311827956989247,
 1.3676470588235294,
 0.8954703832752613,
 3.1]

In [263]:
#creating a dictionary for all team stats in a year for all the teams

def createAnnualDict(year):
    annualDictionary = collections.defaultdict(list)
    for team in teamList:
        team_vector = getAnnualTeamData(team, year)
        annualDictionary[team] = team_vector
    return annualDictionary



In [264]:
createAnnualDict(2019)

defaultdict(list,
            {'Arsenal': [837.0,
              502.0,
              714.0,
              39.0,
              4579.0,
              2832.0,
              14.547085201793722,
              10.710762331838565,
              0.06726457399103139,
              2524.0,
              0.8341044282881692,
              1.198890649762282,
              0.8709926017262639,
              3.6152927120669056],
             'Aston Villa': [369.0,
              518.0,
              605.0,
              22.0,
              3791.0,
              1775.0,
              10.927710843373495,
              13.58433734939759,
              0.02108433734939759,
              1103.0,
              0.6804441702652684,
              1.4696282864913872,
              0.8982910694597575,
              4.392953929539296],
             'Bournemouth': [230.0,
              312.0,
              274.0,
              9.0,
              1675.0,
              959.0,
              11.707182320441989,
       

In [265]:
#creating a training method using the dictionaries defined before. 
#Function will calculate the difference between two teams vectors for a given year. 
#Then the function assigns a yTrain =1 if the hometeam wins and 0 otherwise. 
#The difference vector becomes the xTrain and a label

def getTrainingData(years):
    totalNumGames = 0
    for year in years:
        annual = EPL_data[EPL_data['Year'] == year]
        totalNumGames += len(annual.index)
    numFeatures = len(getAnnualTeamData('Arsenal',2015)) #random team, to find dimensionality
    xTrain = np.zeros(( totalNumGames, numFeatures))
    yTrain = np.zeros(( totalNumGames ))
    indexCounter = 0
    for year in years:
        team_vectors = createAnnualDict(year)
        annual = EPL_data[EPL_data['Year'] == year]
        numGamesInYear = len(annual.index)
        xTrainAnnual = np.zeros(( numGamesInYear, numFeatures))
        yTrainAnnual = np.zeros(( numGamesInYear ))
        counter = 0
        for index, row in annual.iterrows():
            h_team = row['HomeTeam']
            h_vector = team_vectors[h_team]
            a_team = row['AwayTeam']
            a_vector = team_vectors[a_team]
            diff = [a - b for a, b in zip(h_vector, a_vector)]
            if (counter % 2 == 0):
                if len(diff) != 0:
                    xTrainAnnual[counter] = diff
                yTrainAnnual[counter] = 1
            # the opposite of the difference of the vectors should be a true negative, where team 1 does not win
            else:
                if len(diff) != 0:
                    xTrainAnnual[counter] = [ -p for p in diff]
                yTrainAnnual[counter] = 0
            counter += 1
        xTrain[indexCounter:numGamesInYear+indexCounter] = xTrainAnnual
        yTrain[indexCounter:numGamesInYear+indexCounter] = yTrainAnnual
        indexCounter += numGamesInYear
    return xTrain, yTrain

In [266]:
years = range(2018, 2020)
xTrain, yTrain = getTrainingData(years)
np.save('xTrain', xTrain)
np.save('yTrain', yTrain)

In [267]:
xTrain.shape


(4468, 14)

In [268]:
yTrain.shape

(4468,)

In [269]:
lm = linear_model.LinearRegression()

In [270]:
def createGamePrediction(team1_vector, team2_vector, xTrain, yTrain):
    xTrain, X_test, yTrain, Y_test = train_test_split(xTrain, yTrain)
    xTrain.shape, yTrain.shape
    X_test.shape, Y_test.shape
    lm = linear_model.LinearRegression()
    model2 = lm.fit(xTrain, yTrain)
    diff = [[a - b for a, b in zip(team1_vector, team2_vector)]]
    predictions = lm.predict(diff)
    return predictions

In [271]:
team1_vector = getAnnualTeamData("Arsenal", 2020)
team2_vector = getAnnualTeamData("Crystal Palace", 2020)
print (createGamePrediction(team1_vector, team2_vector,xTrain, yTrain))

[0.49856537]


In [272]:
test_data = pd.read_csv('test_data.csv')

In [273]:
test_data.head()

,Game_ID,Year,Date,HomeTeam,AwayTeam
0,1,2020,09-08-19,Liverpool,Norwich
1,2,2020,10-08-19,Bournemouth,Sheffield United
2,3,2020,10-08-19,Burnley,Southampton
3,4,2020,10-08-19,Crystal Palace,Everton
4,5,2020,10-08-19,Leicester,Wolves


In [284]:
def formulatePredictions(n_simulations=3):
    count = 0
    probs = [[0 for x in range(2)] for x in range(len(test_data.index))]
    for index, row in test_data.iterrows():
        game_ID = row['Game_ID']
        year = row['Year'] - 1
        team1_Name = row['HomeTeam']
        team2_Name = row['AwayTeam']
        team1_vector = getAnnualTeamData(team1_Name, year)
        team2_vector = getAnnualTeamData(team2_Name, year)
        prediction = createGamePrediction(team1_vector, team2_vector,xTrain, yTrain)
        probs[index][0] = game_ID
        probs[index][1] = prediction
    probs = pd.np.array(probs)
    count += 1
    return probs

In [285]:
formulatePredictions()

C:\Users\Abhishek\anaconda3\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  


array([[1, array([0.48809961])],
       [2, array([0.48818444])],
       [3, array([0.51835411])],
       [4, array([0.50839845])],
       [5, array([0.46697865])],
       [6, array([0.490895])],
       [7, array([0.46460436])],
       [8, array([0.56092907])],
       [9, array([0.50127868])],
       [10, array([0.533148])],
       [11, array([0.48681253])],
       [12, array([0.48614259])],
       [13, array([0.47386223])],
       [14, array([0.47311976])],
       [15, array([0.47782486])],
       [16, array([0.47311796])],
       [17, array([0.50024425])],
       [18, array([0.4834176])],
       [19, array([0.49711618])],
       [20, array([0.50731536])],
       [21, array([0.48831465])],
       [22, array([0.55850861])],
       [23, array([0.51920329])],
       [24, array([0.50049112])],
       [25, array([0.52045699])],
       [26, array([0.53981048])],
       [27, array([0.52756918])],
       [28, array([0.52339521])],
       [29, array([0.52307656])],
       [30, array([0.5265959

In [277]:
import numpy as np
a = np.asarray(formulatePredictions())
np.savetxt("Predictions_results10.csv", a, delimiter=",")

C:\Users\Abhishek\anaconda3\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  del sys.path[0]
